In [2]:
import gurobipy as gp
from gurobipy import GRB

# Self-contained data (NO file I/O)
Q = 10  # truck capacity (units per truck)

sources = [f"S{i}" for i in range(1, 11)]
destinations = [f"D{j}" for j in range(1, 21)]

supply_vals = [103, 87, 95, 112, 97, 103, 101, 94, 102, 106]  # sum = 1000
demand_vals = [61, 54, 56, 54, 53, 47, 56, 57, 56, 34, 55, 53, 37, 31, 62, 58, 39, 32, 38, 67]  # sum = 1000

# Per-unit costs c_ij (10 x 20), rows correspond to S1..S10, columns to D1..D20
cost_matrix = [
    [2.75, 2.60, 2.90, 1.70, 1.85, 1.98, 2.74, 6.20, 5.75, 6.44, 5.20, 4.54, 5.39, 4.34, 8.28, 8.87, 9.03, 8.49, 9.66, 10.91],
    [5.24, 4.87, 4.72, 4.12, 4.26, 4.55, 4.45, 3.88, 3.24, 4.24, 3.13, 3.16, 3.67, 2.43, 5.83, 6.51, 6.68, 6.10, 7.20, 8.40],
    [5.54, 5.16, 5.09, 4.37, 4.57, 4.78, 4.89, 3.84, 3.08, 4.42, 3.37, 3.53, 3.98, 2.70, 5.84, 6.48, 6.68, 6.14, 7.24, 8.31],
    [4.51, 4.17, 4.01, 3.36, 3.59, 3.73, 3.68, 4.39, 3.93, 4.71, 3.55, 3.15, 3.84, 2.60, 6.51, 7.03, 7.27, 6.69, 7.89, 9.04],
    [4.84, 4.74, 4.77, 3.80, 3.96, 4.10, 4.52, 4.82, 4.03, 5.35, 4.25, 4.14, 4.73, 3.46, 6.89, 7.50, 7.67, 7.20, 8.20, 9.35],
    [9.13, 8.32, 7.78, 7.90, 8.13, 8.45, 7.47, 3.04, 4.25, 2.19, 3.34, 4.07, 3.30, 4.24, 2.77, 2.63, 2.97, 2.56, 3.32, 5.09],
    [8.64, 7.86, 7.34, 7.32, 7.51, 7.83, 7.03, 1.90, 3.14, 1.43, 2.64, 3.64, 2.92, 3.51, 2.41, 2.74, 2.98, 2.46, 3.51, 5.08],
    [9.25, 8.41, 7.81, 8.13, 8.32, 8.60, 7.49, 3.71, 5.00, 2.88, 3.78, 4.27, 3.49, 4.63, 3.62, 3.34, 3.59, 3.29, 3.79, 5.54],
    [10.30, 9.58, 8.89, 9.11, 9.32, 9.69, 8.57, 4.11, 5.29, 3.49, 4.59, 5.28, 4.47, 5.51, 3.22, 2.59, 2.72, 2.68, 2.82, 4.47],
    [7.85, 7.08, 6.57, 6.72, 6.88, 7.18, 6.21, 2.34, 3.56, 1.52, 2.18, 2.78, 2.01, 3.00, 3.47, 3.62, 3.89, 3.41, 4.38, 5.98],
]

# Sanity checks
assert sum(supply_vals) == sum(demand_vals) == 1000
assert len(sources) == 10 and len(destinations) == 20
assert len(cost_matrix) == 10 and all(len(row) == 20 for row in cost_matrix)

# Build parameter dicts
supply = {sources[i]: supply_vals[i] for i in range(10)}
demand = {destinations[j]: demand_vals[j] for j in range(20)}
cost = {(sources[i], destinations[j]): cost_matrix[i][j] for i in range(10) for j in range(20)}

# Model
m = gp.Model("Expanded_Transportation_10x20_NoFiles")

# Decision variables
y = m.addVars(sources, destinations, name="y", lb=0.0, vtype=GRB.CONTINUOUS)  # shipped units
x = m.addVars(sources, destinations, name="x", vtype=GRB.INTEGER)             # trucks (integer)

# Objective: minimize total per-unit transportation cost
m.setObjective(gp.quicksum(cost[i, j] * y[i, j] for i in sources for j in destinations), GRB.MINIMIZE)

# Constraints
# Demand satisfaction
for j in destinations:
    m.addConstr(gp.quicksum(y[i, j] for i in sources) == demand[j], name=f"demand_{j}")

# Supply limits
for i in sources:
    m.addConstr(gp.quicksum(y[i, j] for j in destinations) <= supply[i], name=f"supply_{i}")

# Truck capacity coupling: allow partial loading
for i in sources:
    for j in destinations:
        m.addConstr(y[i, j] <= Q * x[i, j], name=f"cap_{i}_{j}")

# Solve
m.optimize()

# Output
if m.status == GRB.OPTIMAL:
    print(f"Optimal total cost: {m.objVal:.6f}")
    print("\nNonzero shipments (y_ij > 0):")
    for i in sources:
        for j in destinations:
            yval = y[i, j].X
            if yval > 1e-6:
                print(f"  {i}->{j}: y={yval:.2f}, x={int(round(x[i, j].X))}, c_ij={cost[i,j]:.2f}")
else:
    print(f"Optimization ended with status {m.status}")


Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M3 Max
Thread count: 14 physical cores, 14 logical processors, using up to 14 threads

Optimize a model with 230 rows, 400 columns and 800 nonzeros
Model fingerprint: 0x6b16c095
Variable types: 200 continuous, 200 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [1e+00, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+01, 1e+02]
Found heuristic solution: objective 5058.5100000
Presolve removed 200 rows and 200 columns
Presolve time: 0.00s
Presolved: 30 rows, 200 columns, 400 nonzeros
Variable types: 200 continuous, 0 integer (0 binary)

Root relaxation: objective 3.148930e+03, 29 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    3148.9300000 3148.93000  0.00%     -    0s



In [3]:
#32
import gurobipy as gp
from gurobipy import GRB
import numpy as np

# Cost matrix: 12 machines (rows M1..M12) x 12 tasks (columns A..L)
c = np.array([
    [167.4,  98.6, 189.4, 119.6, 182.0, 145.1, 185.4,  94.8, 122.3, 123.3,  96.1,  90.3],
    [156.2,  88.7, 187.3, 124.7, 173.2, 144.3, 179.0,  91.5, 115.1, 119.5, 100.1,  88.6],
    [184.3, 121.0, 216.6, 140.0, 196.2, 168.8, 205.6, 114.2, 133.3, 144.5, 116.0, 107.7],
    [157.9,  92.9, 185.1, 120.3, 175.1, 146.2, 180.8,  86.3, 111.6, 115.9,  98.1,  91.1],
    [175.6, 103.6, 204.5, 130.0, 192.8, 157.5, 194.2, 106.9, 129.9, 134.9, 105.8,  98.6],
    [166.8, 107.0, 199.2, 130.4, 183.6, 159.5, 187.0,  98.2, 121.3, 126.2, 105.9, 101.8],
    [159.7,  93.2, 183.8, 113.0, 171.9, 139.1, 169.6,  85.1, 110.0, 116.7,  90.6,  85.2],
    [184.8, 115.9, 205.1, 138.6, 195.4, 160.1, 200.2, 108.5, 136.9, 140.0, 114.6, 103.9],
    [157.3,  86.2, 186.0, 113.9, 166.2, 136.8, 167.5,  78.8, 107.4, 114.5,  87.2,  78.6],
    [164.8,  97.8, 200.9, 125.8, 188.9, 151.2, 187.7,  99.5, 119.5, 132.1, 101.1,  98.4],
    [164.0,  92.2, 186.2, 115.7, 174.5, 143.0, 175.9,  92.3, 114.0, 121.2,  93.7,  91.2],
    [151.7,  76.7, 179.5, 109.5, 160.6, 128.4, 170.2,  74.4, 103.7, 110.4,  83.7,  75.2]
], dtype=float)

n = c.shape[0]
machines = [f"M{i}" for i in range(1, n + 1)]
tasks = list("ABCDEFGHIJKL")

# Create the model
m = gp.Model("Assignment_12x12")

# Decision variables
x = m.addVars(n, n, vtype=GRB.BINARY, name="x")

# Objective function
m.setObjective(gp.quicksum(c[i, j] * x[i, j] for i in range(n) for j in range(n)), GRB.MINIMIZE)

# Assignment constraints
for i in range(n):
    m.addConstr(gp.quicksum(x[i, j] for j in range(n)) == 1, name=f"row_assign_{i}")
for j in range(n):
    m.addConstr(gp.quicksum(x[i, j] for i in range(n)) == 1, name=f"col_assign_{j}")

# Solve the model
m.optimize()

# Print solution
if m.status == GRB.OPTIMAL:
    print(f"Optimal total cost = {m.objVal:.1f}")
    for i in range(n):
        for j in range(n):
            if x[i, j].X > 0.5:
                print(f"{machines[i]} -> Task {tasks[j]}  (cost {c[i, j]:.1f})")


Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M3 Max
Thread count: 14 physical cores, 14 logical processors, using up to 14 threads

Optimize a model with 24 rows, 144 columns and 288 nonzeros
Model fingerprint: 0x7ae27bf1
Variable types: 0 continuous, 144 integer (144 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [7e+01, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 1609.1000000
Presolve time: 0.00s
Presolved: 24 rows, 144 columns, 288 nonzeros
Variable types: 0 continuous, 144 integer (144 binary)

Root relaxation: objective 1.577600e+03, 39 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    1577.6000000 1577.60000  0.00%     -    0s

Explored 1 nodes (39 simplex iterations) 